# EDA Samples
## OS/Browser 별 이마트앱 접속 통계 (from Bigquery)

### 1. EDA를 위한 python 및 pydatafabric 모듈 준비
Bigquery를 위한 구체적인 emart 패키지 활용 방법은 링크 참고 (https://emartaiflow.github.io/jupyterbook/bigquery.html)

In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pydatafabric.gcp import bq_to_df, bq_to_pandas

warnings.filterwarnings("ignore")

dt='20220401' # 기준 DATE

### 2. GCP 데이터 검색
- GCP에서 제공하는 Data Catalog를 통해 데이터를 검색할 수 있습니다. (https://console.cloud.google.com/datacatalog?referrer=search&project=emart-datafabric&qSystems=BIGQUERY)

### 3. 원천 데이터 확인
pydatafabric 패키지의 bq_to_df 모듈을 활용해 bigquery table을 load하고, 데이터를 확인할 수 있습니다.
> 내부 엔진으로 Apache Spark을 활용합니다.

목표 EDA를 위해 다음 두 테이블을 load하고 조회해 보겠습니다.
- smart-ruler-304409.o2o_log.ST_LOG

우선 ST_LOG 테이블을 조회 해보겠습니다.

In [ ]:
query = f"""
    SELECT
        *
    FROM
        smart-ruler-304409.o2o_log.ST_LOG
    WHERE
        rgst_dt=PARSE_DATE('%Y%m%d', '{dt}')
    LIMIT
        10000
"""

In [ ]:
df = bq_to_df(query)

- ST_LOG 테이블의 스키마를 확인합니다.

In [ ]:
df.printSchema()

In [ ]:
df.filter(df.app_name.isNotNull()).select(df.os_name, df.browser_name, df.cnt).show(10, False)

### 4. 분석 데이터 준비
> 앞서 준비한 테이블을 조회하여 Pandas Dataframe 을 만듭니다.
- 분석에 활용될 데이터 수집을 위해 Query를 작성합니다.

In [ ]:
query = f"""
    SELECT
      os_name,
      browser_name,
      COUNT(*) AS cnt
    FROM
      smart-ruler-304409.o2o_log.ST_LOG
    WHERE
      rgst_dt=PARSE_DATE('%Y%m%d', '{dt}')
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 1000000
"""

- Query를 Bigquery 에서 실행하고, 그 결과를 Pandas Dataframe으로 변환하여 저장합니다.

In [ ]:
pdf = bq_to_pandas(query)

- pandas로 변환된 데이터를 확인합니다.

In [ ]:
pdf.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(pdf.describe())

### 5. OS / Browser 별 접속 분포도
- OS 및 Browser 별 접속 통계를 분포도를 통해 보여줍니다.

In [ ]:
sorted_nb = pdf.groupby(['os_name'])['cnt'].median().sort_values()
fig, ax = plt.subplots(figsize=(5, 5))
sns.boxplot(x=pdf['os_name'], y=pdf['cnt'], order=list(sorted_nb.index))

sorted_nb = pdf.groupby(['browser_name'])['cnt'].median().sort_values()
fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(x=pdf['browser_name'], y=pdf['cnt'], order=list(sorted_nb.index))

- OS 와 Browser 의 접속 통계로 보여줍니다.

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(x="os_name", y="cnt", data=pdf.sort_values(by="cnt", ascending=False))
sns.despine(left=True, bottom=True)

plt.tight_layout()
plt.show()

- OS와 Browser의 데이터 분포 상관관계를 통계로 보여줍니다.

In [ ]:
apps = pdf['os_name'].unique().tolist()

LINE_STYLES = ['solid', 'dashed', 'dashdot', 'dotted']
plt.figure(figsize=(20,8))
plt.title('Histogram of Daily Application Use Count', fontsize=18)

for i, app_name in enumerate(apps):
    sns.distplot(pdf[pdf['os_name'] == app_name]['cnt'], label=app_name, bins=int(180/5),
            hist=False, kde_kws={'linewidth': 2, 'shade':True, 'linestyle':LINE_STYLES[i % 4]})